<a href="https://colab.research.google.com/github/AI-Junction/Histopathologic-Cancer-Detection/blob/master/Histopathologic-Cancer-Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [0]:
#!pip install kaggle

In [2]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

#filename = "/content/.kaggle/kaggle.json"
#filename = "/.kaggle/kaggle.json"
filename = "kaggle.json"
#os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [4]:
!ls -lha kaggle.json



---x-wx--T 1 root root 65 Dec 15 01:42 kaggle.json


In [3]:
os.path.isdir("/root/.kaggle")

False

In [5]:
dir_kaggle = "/root/.kaggle"
if not os.path.isdir(dir_kaggle):
  !mkdir -p ~/.kaggle  
!cp kaggle.json ~/.kaggle/
os.path.isdir(dir_kaggle)

True

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
!kaggle competitions files -c histopathologic-cancer-detection

name                   size  creationDate         
---------------------  ----  -------------------  
sample_submission.csv   2MB  2018-11-15 23:20:36  
train_labels.csv        9MB  2018-11-15 23:20:37  
test.zip                1GB  2018-11-15 23:21:06  
train.zip               5GB  2018-11-15 23:23:49  


In [10]:
os.path.isdir("drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection")

True

In [14]:
os.path.isfile("drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train_labels.csv.zip")

True

In [17]:
os.path.isdir("drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train")

True

In [0]:
if not os.path.isdir("drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection"):
  ! mkdir -p "drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection"



In [0]:
if not os.path.isfile("drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train_labels.csv.zip"):
  !kaggle competitions download -c histopathologic-cancer-detection -f sample_submission.csv -p "/content/drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection"
  !kaggle competitions download -c histopathologic-cancer-detection -f train_labels.csv -p "/content/drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection"
  !kaggle competitions download -c histopathologic-cancer-detection -f test.zip -p "/content/drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection"
  !kaggle competitions download -c histopathologic-cancer-detection -f train.zip -p "/content/drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection"


In [0]:
if not os.path.isfile("drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train_labels.csv"):
  !unzip "drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train_labels.csv.zip" -d "drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection"

In [0]:
'''
if not os.path.isdir("drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/test"):
  !mkdir -p "drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/test"
  !unzip "drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/test.zip" -d "drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/test"
'''  

In [0]:
from numpy.random import seed
seed(101)
from tensorflow import set_random_seed
set_random_seed(101)

import pandas as pd
import numpy as np


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

import os
import cv2

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
IMAGE_SIZE = 96
IMAGE_CHANNELS = 3

SAMPLE_SIZE = 80000 # the number of images we use from each of the two classes

In [24]:
os.listdir('drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection')

['sample_submission.csv.zip',
 'train_labels.csv.zip',
 'test.zip',
 'train.zip',
 'train',
 'train_labels.csv']

In [25]:
print(len(os.listdir('drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train')))
#print(len(os.listdir('../input/test')))

38240


In [33]:
os.path.isfile('drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train_labels.csv')

True

In [0]:
!chmod 600 'drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train_labels.csv'

In [39]:
ls -l 'drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train_labels.csv'

-rw------- 1 root root 9461084 Nov 15 23:48 'drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train_labels.csv'


In [40]:
df_data = pd.read_csv('drive/My Drive/All ML Datasets/Histopathologic-Cancer-Detection/train_labels.csv', engine='python')

# removing this image because it caused a training error previously
df_data[df_data['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']

# removing this image because it's black
df_data[df_data['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']


print(df_data.shape)

(220025, 2)


In [41]:
df_data['label'].value_counts()

0    130908
1     89117
Name: label, dtype: int64